In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import tensorflow as tf

Assuming we achieved pretty good accuracy during the training and testing phases, we can now use the trained model for inference &mdash; in other words, to predict the classification of images that the network has never seen before.

But before we move on, we'll load the code you've already seen in previous notebooks:

In [9]:
!wget -Nq https://raw.githubusercontent.com/MicrosoftDocs/tensorflow-learning-path/main/intro-keras/kintro.py
from kintro import *

Remember that in the previous notebook, after successfully training our network, we saved the weights of the model. We can now reconstruct a trained model using those weights.

In [10]:
model = NeuralNetwork()
model.load_weights('outputs/weights').expect_partial()

Making a prediction is easy &mdash; we simply call the model's `predict` method and pass it some images. The true power of a neural network is its ability to make predictions for data that has not been seen during training or testing. But for the sake of simplicity, we'll demonstrate how prediction works by re-using a few of the images that we used during testing. The code below retrieves the first three images in our test data and uses them to make predictions. We then print the actual and predicted labels.

In [11]:
batch_size = 64

(_, test_dataset) = get_data(batch_size)
(X_batch, actual_index_batch) = test_dataset.as_numpy_iterator().next()
X = X_batch[0:3, :, :]
actual_indices = actual_index_batch[0:3]

predictions = model.predict(X)

print('\nPredicting:')
for (actual_index, prediction) in zip(actual_indices, predictions):
  actual_name = labels_map[actual_index]
  predicted_index = tf.math.argmax(prediction).numpy()
  predicted_name = labels_map[predicted_index]
  print(f'Actual: {actual_name}, Predicted: {predicted_name}')


Predicting:
Actual: Bag, Predicted: Dress
Actual: Sneaker, Predicted: Sneaker
Actual: Ankle Boot, Predicted: Ankle Boot


With only two epochs of training, our network isn't very accurate, but it should get at least two out of three predictions right. You can increase the number of epochs in the training phase to get better predictions.

Note that you can also get probabilities of the input image being of a certain class, in which case we need to normalize the output of our network using `softmax` to get probabilities. Here are the predictions for the first image in our test set: 

In [18]:
probs = tf.nn.softmax(predictions[0])
for i,p in enumerate(probs):
    print(f'{labels_map[i]} -> {p:.3f}')

T-Shirt -> 0.102
Trouser -> 0.002
Pullover -> 0.001
Dress -> 0.870
Coat -> 0.001
Sandal -> 0.000
Shirt -> 0.023
Sneaker -> 0.000
Bag -> 0.001
Ankle Boot -> 0.000


If you need to compute probabilities often, you can specify `activation='softmax'` for the final `Dense` layer of your network. In this case the network would give you probabilities as output, and you need to omit `use_logits=True` in the `SparseCategoricalCrossentropy` loss function. 